In [ ]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from gotmtool import *
from lesview import *
from sbl_bbl import *

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2'
turbmethods = ['SMCLT-H15', 'SMCLT-KC04', 'KPPLT-LF17', 'KPPLT-LF17-E']

turbmethod = 'SMCLT-H15'
runs = {'r1': '', 'r2': '_rf'}
ds_gotm = {}
ds_ocgn = {}
for rkey in runs.keys():
    datapath = os.path.join(os.path.pardir, 'tests', '{:s}{:s}'.format(casename, runs[rkey]))
    filepath = os.path.join(datapath, 'averages.jld2')
    data_pfl = OceananigansDataProfile(filepath=filepath)
    ds_ocgn[rkey] = data_pfl.dataset
    tmp = {}
    gotm_dir = os.path.join(os.path.pardir, 'gotm', 'run', '{:s}{:s}'.format(casename, runs[rkey]))
    for turbmethod in turbmethods:
        gotm_sim = Simulation(path=os.path.join(gotm_dir, '{:s}-Rlx'.format(turbmethod)))
        tmp[turbmethod] = gotm_sim.load_data()
    ds_gotm[rkey] = tmp

In [ ]:
figpath  = 'overview_{:s}'.format(casename)
os.makedirs(figpath, exist_ok=True)

In [ ]:
g = 9.81
H = 30
N2 = 1.962e-4
bstar = N2 * H
u10 = 8
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
alphaT = 2.0e-4
lat = 45
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)
Ti = inertial_period(lat)
print('Ti = {:6.3f}'.format(Ti))

In [ ]:
tslice = {}
for rkey in runs.keys():
    print('---- Oceananigans {:s} ----'.format(rkey))
    N2_ocgn = nondim_da(ds_ocgn[rkey].data_vars['b'].differentiate(coord='z')/N2, H=H, Tf=Ti)
    tslice[rkey] = get_tslice(N2_ocgn, Ti)

In [ ]:
def get_das_ocgn(ds_ocgn):
    das_ocgn = dict(
        wb = (ds_ocgn.data_vars['wb']+ds_ocgn.data_vars['wbsb'])/ustar/bstar*10**3,
        wu = (ds_ocgn.data_vars['wu']+ds_ocgn.data_vars['wusb'])/ustar**2,
        wv = (ds_ocgn.data_vars['wv']+ds_ocgn.data_vars['wvsb'])/ustar**2,
        )
    return das_ocgn

def get_das_gotm(ds):
    das_gotm = dict(
        wb = get_flux(ds.data_vars['temp'].squeeze(),
                      ds.data_vars['nuh'].squeeze(),
                      ds.data_vars['gamh'].squeeze())*alphaT*g/ustar/bstar*10**3,
        wu = (get_flux(ds.data_vars['u'].squeeze(),
                      ds.data_vars['num'].squeeze(),
                      ds.data_vars['gamu'].squeeze())
              -ds.data_vars['nucl'].squeeze()*ds.data_vars['dusdz'].squeeze())/ustar**2,
        wv = (get_flux(ds.data_vars['v'].squeeze(),
                      ds.data_vars['num'].squeeze(),
                      ds.data_vars['gamv'].squeeze())
              -ds.data_vars['nucl'].squeeze()*ds.data_vars['dvsdz'].squeeze())/ustar**2,
    )
    return das_gotm
def get_mflux_gotm(ds):
    das_gotm = dict(
        wuel = get_flux(ds.data_vars['u'].squeeze(),
                        ds.data_vars['num'].squeeze(),
                        ds.data_vars['gamu'].squeeze())/ustar**2,
        wvel = get_flux(ds.data_vars['v'].squeeze(),
                        ds.data_vars['num'].squeeze(),
                        ds.data_vars['gamv'].squeeze())/ustar**2,
        wust = -ds.data_vars['nucl'].squeeze()*ds.data_vars['dusdz'].squeeze()/ustar**2,
        wvst = -ds.data_vars['nucl'].squeeze()*ds.data_vars['dvsdz'].squeeze()/ustar**2,
    )
    return das_gotm

In [ ]:
labels = dict(
    wb = '$10^3\overline{w^\prime b^\prime}/u_*b_*$',
    wu = '$\overline{w^\prime u^\prime}/u_*^2$',
    wv = '$\overline{w^\prime v^\prime}/u_*^2$',
)
turbmethods = ['SMCLT-H15', 'SMCLT-KC04']
colors = ['tab:blue', 'tab:red']
linestyles = ['-', '-']
kw_line_les  = dict(linestyle='-', color='k')
kw_line_gotm = {}
for i, turbmethod in enumerate(turbmethods):
    kw_line_gotm[turbmethod] = dict(linestyle=linestyles[i], color=colors[i])
comps = ['el', 'st']
lstyles = ['--', ':']
kw_line_mflux = {}
for i, comp in enumerate(comps):
    kw_line_mflux[comp] = dict(linestyle=lstyles[i], color=colors[0])
    
abc = dict(
    wb = 'adgj',
    wu = 'behk',
    wv = 'cfil',
)
abc_loc = dict(
    wb = ['left',  'left',  'left',  'left'],
    wu = ['right', 'right', 'left',  'left'],
    wv = ['right', 'right', 'right', 'right'],
)
abc_x = {'left': 0.1, 'right': 0.9}

titles = ['T1', 'T2']
tags = ['Aligned', 'Opposite']

fig, axarr = plt.subplots(3, 4, sharey='row')
fig.set_size_inches([9, 8])
for k, rkey in enumerate(runs.keys()):
    das_ocgn = nondim_das(get_das_ocgn(ds_ocgn[rkey]), H=H, Tf=Ti)
    das_gotm = {}
    das_gotm_mflux = {}
    for turbmethod in turbmethods:
        ds = ds_gotm[rkey][turbmethod]
        das_gotm[turbmethod] = nondim_das(get_das_gotm(ds), H=H, Tf=Ti)
        if turbmethod == 'SMCLT-H15':
            das_gotm_mflux = nondim_das(get_mflux_gotm(ds), H=H, Tf=Ti)
    for j in np.arange(2):
        for i, var in enumerate(das_ocgn.keys()):
            ax = axarr[i, j+2*k]
            da = das_ocgn[var].sel(time=tslice[rkey][j]).mean(dim='time')
            da.plot(ax=ax, y=da.dims[0], **kw_line_les, label='LES')
            for turbmethod in turbmethods:
                da = das_gotm[turbmethod][var].sel(time=tslice[rkey][j]).mean(dim='time')
                da.plot(ax=ax, y=da.dims[0], **kw_line_gotm[turbmethod], label=turbmethod[6:])
            ax.text(abc_x[abc_loc[var][k*2+j]], 0.1, '({:s})'.format(abc[var][k*2+j]), transform=ax.transAxes, va='bottom', ha=abc_loc[var][k*2+j])
            if i == 0:
                ax.set_title(titles[j], fontsize=10)
                if j == 1:
                    ax.text(-0.1, 1.1, tags[k], transform=ax.transAxes, va='bottom', ha='center')
                    if k == 0:
                        ax.legend(loc='lower right', fontsize=9)
            else:
                for comp in comps:
                    da = das_gotm_mflux[var+comp].sel(time=tslice[rkey][j]).mean(dim='time')
                    da.plot(ax=ax, y=da.dims[0], **kw_line_mflux[comp])
                ax.set_title('')
            ax.set_ylabel('')
            ax.set_ylim([-1, 0])
            ax.set_xlabel(labels[var])
    axarr[0,2*k+1].set_xlim([-1,1])

for i in np.arange(3):
    ax = axarr[i,0]
    ax.set_ylabel('$z/H$')

lglstyles = ['-']+lstyles
ax = axarr[0,3]
lns = []
for i, ls in enumerate(lglstyles):
    ln, = ax.plot(np.nan, np.nan, linestyle=ls, color='tab:blue')
    lns.append(ln)
ax.legend(lns, ['Total', 'Eulerian', 'Stokes'], loc='lower right', fontsize=9)

plt.subplots_adjust(top=0.94, bottom=0.08, left=0.08, right=0.97, hspace=0.35, wspace=0.15)
figname = os.path.join(figpath, 'profiles_relax_smclt')
fig.savefig(figname, dpi = 300, facecolor='w')

In [ ]:
labels = dict(
    wb = '$10^3\overline{w^\prime b^\prime}/u_*b_*$',
    wu = '$\overline{w^\prime u^\prime}/u_*^2$',
    wv = '$\overline{w^\prime v^\prime}/u_*^2$',
)
turbmethods = ['KPPLT-LF17', 'KPPLT-LF17-E']
colors = ['tab:purple', 'tab:purple']
linestyles = ['-', '--']
kw_line_les  = dict(linestyle='-', color='k')
kw_line_gotm = {}
for i, turbmethod in enumerate(turbmethods):
    kw_line_gotm[turbmethod] = dict(linestyle=linestyles[i], color=colors[i])

abc = dict(
    wb = 'adgj',
    wu = 'behk',
    wv = 'cfil',
)
abc_loc = dict(
    wb = ['left',  'left',  'left',  'left'],
    wu = ['left',  'right', 'right', 'right'],
    wv = ['left',  'left',  'left',  'left'],
)
abc_x = {'left': 0.1, 'right': 0.9}

titles = ['T1', 'T2']
tags = ['Aligned', 'Opposite']

fig, axarr = plt.subplots(3, 4, sharey='row')
fig.set_size_inches([9, 8])
for k, rkey in enumerate(runs.keys()):
    das_ocgn = nondim_das(get_das_ocgn(ds_ocgn[rkey]), H=H, Tf=Ti)
    das_gotm = {}
    das_gotm_mflux = {}
    for turbmethod in turbmethods:
        ds = ds_gotm[rkey][turbmethod]
        das_gotm[turbmethod] = nondim_das(get_das_gotm(ds), H=H, Tf=Ti)
        das_gotm_mflux[turbmethod] = nondim_das(get_mflux_gotm(ds), H=H, Tf=Ti)
    for j in np.arange(2):
        for i, var in enumerate(das_ocgn.keys()):
            ax = axarr[i, j+2*k]
            da = das_ocgn[var].sel(time=tslice[rkey][j]).mean(dim='time')
            da.plot(ax=ax, y=da.dims[0], **kw_line_les, label='LES')
            for turbmethod in turbmethods:
                da = das_gotm[turbmethod][var].sel(time=tslice[rkey][j]).mean(dim='time')
                da.plot(ax=ax, y=da.dims[0], **kw_line_gotm[turbmethod], label=turbmethod[6:])
            ax.set_ylabel('')
            ax.set_ylim([-1, 0])
            ax.set_xlabel(labels[var])
            ax.text(abc_x[abc_loc[var][k*2+j]], 0.9, '({:s})'.format(abc[var][k*2+j]), transform=ax.transAxes, va='top', ha=abc_loc[var][k*2+j])
            if i == 0:
                ax.set_title(titles[j], fontsize=10)
                if j == 1:
                    ax.text(-0.1, 1.1, tags[k], transform=ax.transAxes, va='bottom', ha='center')
                    if k == 0:
                        ax.legend(loc='lower right', fontsize=9)
            else:
                ax.set_title('')
    axarr[0,2*k+1].set_xlim([-1,1])

for i in np.arange(3):
    ax = axarr[i,0]
    ax.set_ylabel('$z/H$')
    
plt.subplots_adjust(top=0.94, bottom=0.08, left=0.08, right=0.97, hspace=0.35, wspace=0.15)
figname = os.path.join(figpath, 'profiles_relax_kpplt')
fig.savefig(figname, dpi = 300, facecolor='w')